In [48]:
import torch
from transformers import AutoTokenizer
from tqdm import tqdm
import os
import json
import sys
from training_with_sentence_transformers.models import ColBERTTransformer
import gzip


#MODEL_DIR=colbert_splade_distill_num1_updatemrr2000_kldiv_ib_position_focal_gamma5.0-alpha1.0-ibp0.1_denoiseFalse_num20_kldiv_ib_position_focal5-lr1e-05-batch_size_16x2-2023-01-20
#python inference_ColBERT.py training_with_sentence_transformers/output/$MODEL_DIR/42000/0_ColBERTTransformer training_with_sentence_transformers/output/$MODEL_DIR/index_42k

agg = "max"
bsize = 128
model_type_or_dir = "training_with_sentence_transformers/output/colbert_cont_128_num20_kldiv_ib5-lr1e-05-batch_size_32x1-2023-04-17/236000/0_ColBERTTransformer" #"colspla-prf_from_colbert_3e-6_negpersys5" #"output/0_MLMTransformer"

# loading model and tokenizer
dense_weight = 0.0
model = ColBERTTransformer(model_type_or_dir, max_seq_length=256, dim=128)
checkpoint = torch.load(os.path.join(model_type_or_dir, "checkpoint.pt"), map_location='cpu')
model.load_state_dict(checkpoint)
    
model.eval()
model.to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_type_or_dir)
tokens = ["[unused0]", "[unused1]", "[unused2]"] #[unused0] for query, [unused1] for doc, [unused2] for query expansion
tokenizer.add_tokens(tokens, special_tokens=True)
reverse_voc = {v: k for k, v in tokenizer.vocab.items()}
             

In [2]:
with open("../msmarco/collection.tsv") as f:
    for line in tqdm(f):
        did, doc = line.strip().split("\t")    
        break

0it [00:00, ?it/s]


In [3]:
with torch.no_grad():
    d_features = tokenizer("[unused1] " + doc, return_tensors="pt", truncation=True).to('cuda')

In [5]:
with torch.no_grad():
    doc_rep = model(d_features)
d_mask = doc_rep['attention_mask'].to('cuda')
d_mask = d_mask.unsqueeze(-1)
d_emb = doc_rep['last_layer_embeddings']
token_rep_d =  d_emb * d_mask

In [39]:
token_rep_d.shape

torch.Size([1, 56, 128])

In [72]:
d_features['input_ids']

tensor([[  101,     2,  1996,  3739,  1997,  4807, 13463,  4045,  9273,  2001,
          8053,  2590,  2000,  1996,  3112,  1997,  1996,  7128,  2622,  2004,
          4045, 24823,  2001,  1012,  1996,  2069,  6112,  5689,  2058,  1996,
          8052,  6344,  1997,  1996,  9593,  6950,  1998,  6145,  2003,  2054,
          2037,  3112,  5621,  3214,  1025,  5606,  1997,  5190,  1997,  7036,
          3268, 27885, 22779,  9250,  1012,   102]], device='cuda:0')

In [93]:
## another way when we deal with static embedding

non_context = [model({'input_ids':torch.tensor([[101, 2, tok, 102]]).to('cuda'), 'token_type_ids':  torch.tensor([[0,0,0,0]]).to('cuda'), 
       'attention_mask': torch.tensor([[1,1,1,1]]).to('cuda')})['last_layer_embeddings'][:,2,:] for tok in d_features['input_ids'].tolist()[0]]

non_context = torch.stack(non_context,1)


In [98]:
[torch.dot(torch.nn.functional.normalize(non_context[0])[i], torch.nn.functional.normalize(token_rep_d[0])[i]) for i in range(non_context.shape[1])]

[tensor(0.1446, device='cuda:0', grad_fn=<DotBackward0>),
 tensor(0.9286, device='cuda:0', grad_fn=<DotBackward0>),
 tensor(0.2465, device='cuda:0', grad_fn=<DotBackward0>),
 tensor(0.6897, device='cuda:0', grad_fn=<DotBackward0>),
 tensor(0.3179, device='cuda:0', grad_fn=<DotBackward0>),
 tensor(0.7382, device='cuda:0', grad_fn=<DotBackward0>),
 tensor(0.5947, device='cuda:0', grad_fn=<DotBackward0>),
 tensor(0.7179, device='cuda:0', grad_fn=<DotBackward0>),
 tensor(0.6965, device='cuda:0', grad_fn=<DotBackward0>),
 tensor(0.3768, device='cuda:0', grad_fn=<DotBackward0>),
 tensor(0.8051, device='cuda:0', grad_fn=<DotBackward0>),
 tensor(0.7754, device='cuda:0', grad_fn=<DotBackward0>),
 tensor(0.3616, device='cuda:0', grad_fn=<DotBackward0>),
 tensor(0.2756, device='cuda:0', grad_fn=<DotBackward0>),
 tensor(0.7610, device='cuda:0', grad_fn=<DotBackward0>),
 tensor(0.2471, device='cuda:0', grad_fn=<DotBackward0>),
 tensor(0.3046, device='cuda:0', grad_fn=<DotBackward0>),
 tensor(0.7783

In [82]:
[torch.dot(torch.tensor(doc_emb[0,i]), torch.tensor(non_context[i])) for i in range(len(doc_emb))]

tensor(-0.0174, device='cuda:0', grad_fn=<SumBackward0>)

In [78]:
torch.sum(torch.nn.functional.normalize(non_context[:,2,:]) * torch.nn.functional.normalize(token_rep_d[:,4,:]))

tensor(0.3179, device='cuda:0', grad_fn=<SumBackward0>)

In [96]:
doc_norm = torch.nn.functional.normalize(token_rep_d, dim = 2)
static_norm = torch.nn.functional.normalize(non_context[:,1:2,:], dim = 2)

In [42]:
torch.sum(doc_norm[0,1,:] * static_norm)

tensor(-0.1150, device='cuda:0', grad_fn=<SumBackward0>)

In [43]:
torch.norm(doc_norm, dim=2)

tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000]], device='cuda:0')

In [45]:
static_norm

tensor([[[-5.3690e-02,  5.6263e-02,  1.6762e-03, -7.0259e-02,  6.0122e-05,
           1.0090e-02, -9.4712e-02, -1.2329e-01,  7.4517e-02, -2.5461e-02,
           2.4914e-02,  1.7607e-02,  1.1017e-01,  1.3662e-02,  2.3778e-02,
           1.2803e-01, -2.0093e-01,  4.1182e-02, -9.9289e-02,  2.4423e-02,
           1.6703e-01, -2.1474e-03,  4.7846e-02,  5.0802e-02,  8.6490e-02,
           3.4477e-02,  3.1344e-02,  1.0562e-01, -1.3178e-01,  4.9349e-02,
          -1.5922e-02, -2.5611e-02, -2.2045e-02,  1.1995e-01, -4.0245e-02,
           1.2449e-01, -1.1227e-01,  8.9956e-02,  1.1854e-02,  1.3891e-01,
          -6.7069e-02, -1.0372e-01, -3.1512e-02, -5.2647e-02,  9.2421e-02,
          -1.2634e-02,  1.0818e-02,  7.6936e-02,  4.0735e-02, -5.1563e-02,
          -2.4714e-02, -2.9732e-02, -7.4961e-02, -2.0589e-03, -1.5524e-01,
           7.7767e-02, -1.4685e-02,  7.5426e-03, -1.1453e-02,  6.6873e-02,
          -9.3952e-02, -7.2675e-02, -6.1948e-02, -3.4644e-02, -7.5766e-02,
           2.5492e-02,  8

In [32]:
doc_norm.shape

torch.Size([1, 56, 128])